# Chroma DB

In [1]:
from langchain_chroma import Chroma
from langchain.retrievers import EnsembleRetriever
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

In [2]:
emb = OpenAIEmbeddings(model="text-embedding-3-small")

In [3]:
sv_2024_pdf_path = "../data/Sustainability_report_2024_kr.pdf"
sv_2025_pdf_path = "../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf"

In [4]:
# 문서 가져오기
loader_2024 = PyPDFLoader(sv_2024_pdf_path)
docs_2024 = loader_2024.load()
print("2024 문서 로드 완료")

loader_2025 = PyPDFLoader(sv_2025_pdf_path)
docs_2025 = loader_2025.load()
print("2025 문서 로드 완료")

2024 문서 로드 완료
2025 문서 로드 완료


In [5]:
print(len(docs_2024))
print(len(docs_2025))

83
87


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

chunk_2024 = splitter.split_documents(docs_2024)
chunk_2025 = splitter.split_documents(docs_2025)

In [7]:
# vectorstore 에 저장
vs_2024 = Chroma.from_documents(
    documents = chunk_2024,
    collection_name = "samsung_2024",
    persist_directory = "./chroma_store",
    embedding = emb
)

vs_2025 = Chroma.from_documents(
    documents = chunk_2025,
    collection_name = "samsung_2025",
    persist_directory = "./chroma_store",
    embedding = emb
)

In [8]:
r_2024 = vs_2024.as_retriever(search_kwargs={"k" : 5})
r_2025 = vs_2025.as_retriever(search_kwargs={"k" : 5})

In [9]:
r_2024.invoke("2024년 삼성 경영 관련 내용 알려줘")

[Document(id='41bd973e-bd24-4998-9d56-052a7ddb02d0', metadata={'creationdate': '2024-11-25T11:10:32+09:00', 'source': '../data/Sustainability_report_2024_kr.pdf', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'trapped': '/False', 'page_label': '30', 'producer': 'Adobe PDF Library 15.0', 'moddate': '2024-11-25T11:10:46+09:00', 'page': 29, 'total_pages': 83}, page_content='삼성전자 지속가능경영보고서 2024 30Our Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People\n사회적 책임을 다하며 미래로 함께 나아갑니다.\nPeople\n31    임직원\n39    공급망\n45    사회공헌\n48    개인정보보호/보안\n50    고객의 안전/품질'),
 Document(id='370fe9e5-20b8-4c10-b520-d4ebc8f3c24a', metadata={'creationdate': '2024-11-25T11:10:32+09:00', 'page_label': '82', 'total_pages': 83, 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'source': '../data/Sustainability_report_2024_kr.pdf', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'page': 81, 'producer': 'Adobe PDF Library 15.0'}, page_content='삼성전자 지속가능경영보고서 2024\n82\nOur Company Appen

In [10]:
r_2025.invoke("2025년 삼성 경영 관련 내용 알려줘")

[Document(id='58aedfbe-8d11-4d92-852e-943e1bd7d9fd', metadata={'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'producer': 'Adobe PDF Library 15.0', 'trapped': '/False', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'moddate': '2025-09-04T16:51:11+09:00', 'creationdate': '2025-07-10T16:11:16+09:00', 'page_label': '5', 'total_pages': 87, 'page': 4}, page_content='삼성전자 지속가능경영보고서 2025\n05\nOur Company AppendixFacts & Figures PrinciplePlanet People\n회사소개\nAbout Us\n삼성전자주식회사(이하 삼성전자)는 인재와 기술을 기반으로 최고의 제품과 서비스를 창출함으로써 인류사회에 기여하는 글로벌 초일류 기업을 지향합니다. 이를 위해 삼성전자가 지켜나갈 약속인 5가지 경영원칙 을 세부원칙과 \n행동지침으로 구체화하고, 삼성전자 임직원이 지켜야 할 행동규범 으로 제정하여 모든 경영활동의 기준으로 삼고 있습니다. 앞으로도 삼성전자는 조직문화에 5가지 핵심가치 를 내재화하여 지속적으로 성장해갈 것입니다.\n사업부문 및 글로벌 네트워크 소개\n삼성전자는 제품 특성에 따라 DX(Device eXperience)와 DS(Device Solutions) 2개 부문으로 나뉘어 독립적으로 운영되고 있습니다. DX부문은 스마트폰, 네트워크 시스템, 컴퓨터, TV, 냉장고, 세탁기, 에어컨, 의료기기 등 완제품을 \n생산·판매하고 있으며, DS부문은 메모리 사업, Foundry 사업, System LSI 사업으로 구성되어 DRAM, NAND Flash, 모바일AP 등의 반도체 부품을 생산

In [11]:
ensemble_r = EnsembleRetriever(retrievers=[r_2024, r_2025], weights=[0.3, 0.7])
result = ensemble_r.invoke("2024년과 2025년 차이 요약해줘")
print(result[0].page_content)

삼성전자 지속가능경영보고서 2025
62
경제성과
핵심 재무 성과
1)
2022년 2023년 2024년
매출액 조 원  302.2  258.9  300.9 
영업이익 조 원  43.4  6.6  32.7 
당기순이익 조 원  55.7  15.5  34.5 
1) 연결 재무제표 기준
사업부문별 매출
1)
2022년 2023년 2024년
[사업부문별 매출(절대값)]
DX부문 조 원  182.5  170.0  174.9 
DS부문 조 원  98.5  66.6  111.1 
SDC 조 원  34.4  31.0  29.2
Harman
2)
조 원  13.2  14.4  14.3 
[사업부문별 매출(비율)]
DX부문 %  56  60  53 
DS부문 %  30  24  34 
SDC %  10  11  9 
Harman %  4  5  4 
1) 순매출액 기준
2) 2016년 인수
지역별 매출
1)
2022년 2023년 2024년
[지역별 매출(절대값)]
미주 조 원  119.0  92.1  118.8 
유럽 조 원  50.3  48.1  50.1 
한국 조 원  48.7  45.6  39.8 
아시아·아프리카
2) 조 원  84.3  73.1  92.1 
[지역별 매출(비율)]
미주 %  39  35  39 
유럽 %  17  19  17 
한국 %  16  18  13 
아시아∙아프리카 %  28  28  31 
1) 순매출액 기준
2) 2024년 보고서부터 아시아·아프리카에 중국도 합산
경제 가치 분배 2022년 2023년 2024년
[협력회사] 
구매 비용 조 원  219.8  212.8  226.1 
[지역사회]
사회공헌 비용 조 원  0.4  0.4  0.3 
[주주·투자자] 
배당금 조 원  9.8  9.8  9.8 
배당성향 %  18  68  29 
[채권자] 
이자 비용 조 원  0.8  0.9  0.9 
[임직원] 
인건비 조 원  37 .6  38.0  40.5 
[정부]
조세공과금 조 원  13.0  8.2  8.2


### Chroma DB 병합

In [12]:
import chromadb
from chromadb.config import Settings

client = chromadb.PersistentClient(path="./chroma_store")

src24 = client.get_collection("samsung_2024")   # get_collection : 데이터베이스 로드
src25 = client.get_collection("samsung_2025")

In [13]:
# 새롭게 전체 데이터베이스 생성
vs_all = Chroma(
    collection_name = "samsung_all",
    persist_directory = "./chroma_store",
    embedding_function = emb
)

In [14]:
dst = client.get_collection("samsung_all")

In [15]:
# 2. samsung_2024 → samsung_all 복사
batch = 1000
offset = 0
while True:
    batch_data = src24.get(
        include=["embeddings", "metadatas", "documents"],
        limit=batch, offset=offset
    )
    ids = batch_data["ids"]
    if not ids:
        break

    # ID 충돌 방지: 접두사 붙이기
    new_ids = [f"{src24.name}::{i}" for i in ids]

    dst.upsert(
        ids=new_ids,
        documents=batch_data["documents"],
        embeddings=batch_data["embeddings"],   # 이미 계산된 벡터 재사용
        metadatas=batch_data["metadatas"],
    )
    offset += len(ids)
    print(f"[{src24.name}] moved {offset} docs so far...")

[samsung_2024] moved 207 docs so far...


In [16]:
# 3. samsung_2025 → samsung_all 복사
batch = 1000
offset = 0
while True:
    batch_data = src25.get(
        include=["embeddings", "metadatas", "documents"],
        limit=batch, offset=offset
    )
    ids = batch_data["ids"]
    if not ids:
        break

    new_ids = [f"{src25.name}::{i}" for i in ids]

    dst.upsert(
        ids=new_ids,
        documents=batch_data["documents"],
        embeddings=batch_data["embeddings"],
        metadatas=batch_data["metadatas"],
    )
    offset += len(ids)
    print(f"[{src25.name}] moved {offset} docs so far...")

[samsung_2025] moved 237 docs so far...


In [17]:
vs_all._collection.count()

444

In [18]:
vs_all.similarity_search("올해와 작년 변화 요약", k=5)

[Document(id='samsung_2024::b5979d39-87a3-4be0-8d4b-fe0431a72320', metadata={'creationdate': '2024-11-25T11:10:32+09:00', 'total_pages': 83, 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'page_label': '56', 'page': 55, 'moddate': '2024-11-25T11:10:46+09:00', 'source': '../data/Sustainability_report_2024_kr.pdf', 'trapped': '/False', 'producer': 'Adobe PDF Library 15.0'}, page_content='삼성전자 지속가능경영보고서 2024\n56\nOur Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People\n경제성과\n핵심 재무 성과\n1)\n2021년 2022년 2023년\n매출액 조 원  279.6  302.2  258.9 \n영업이익 조 원  51.6  43.4 6.6\n당기순이익 조 원  39.9  55.7 15.5\n1) 연결 재무제표 기준\n사업부문별 매출\n1)\n2021년 2022년 2023년\n사업부문별 매출(절대값)\nDX부문 조 원  166.3  182.5 170.0\nDS부문 조 원  95.4  98.5 66.6\nSDC 조 원  31.7  34.4 31\nHarman 조 원  10.0  13.2 14.4\n사업부문별 매출(비율)\nDX부문 %  55  56 60\nDS부문 %  31  30 24\nSDC %  11  10 11\nHarman %  3  4 5\n1) 순매출액 기준\n지역별 매출\n1)\n2021년 2022년 2023년\n지역별 매출(절대값)\n미주 조 원  97.9  119.0  92.1 \n유럽 조 원  50.3  50.3  48.1 \n한국 조 원  